<a href="https://colab.research.google.com/github/pcsilcan/aed/blob/master/20202/aed_20202_0901_comparing_sorts_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
%%writefile 1.cpp
#include <algorithm>
#include <chrono>
#include <iomanip>
#include <iostream>
#include <string>
#include <vector>

#include <stdlib.h>
#include <time.h>

using namespace std;
using namespace std::chrono;

using uint=unsigned;

template <typename T>
ostream& operator<<(ostream& os, vector<T> a) {
    os << "[ ";
    for (uint e : a) {
        os << e << " ";
    }
    os << "]";
    return os;
}

void shuffle(vector<float>& a) {
    for (uint i = a.size() - 1; i > 0; --i) {
        int randomPos = rand() % i;
        swap(a[i], a[randomPos]);
    }
}

bool isSorted(vector<float>& a) {
    for (uint i = 0; i < a.size() - 1; ++i) {
        if (a[i] > a[i+1]) return false;
    }
    return true;
}

void normalRandom(vector<float>& a) {
    for (uint i = 0; i < a.size(); ++i) {
        a[i] = (i + 1) * 10;
    }
    srand(12); // queremos que sea el mismo arreglo para todos los sorts
    shuffle(a);
}

void reverse(vector<float>& a) {
    float max = a.size() * 10;
    for (uint i = 0; i < a.size(); ++i) {
        a[i] = max - i * 10;
    }
}

void fewUnique(vector<float>& a) {
    for (uint i = 0; i < a.size(); ++i) {
        a[i] = (i % 20 + 1) * 10;
    }
    srand(12); // queremos que sea el mismo arreglo para todos los sorts
    shuffle(a);
}

void almostSorted(vector<float>& a) {
    for (uint i = 0; i < a.size(); ++i) {
        a[i] = (i + 1) * 10 + rand() % 30 - 15;
    }
}

void exchangeSort(vector<float>& a) {
    for (uint i = 0; i < a.size() - 1; ++i) {
        for (uint j = i + 1; j < a.size(); ++j) {
            if (a[i] > a[j]) {
                swap(a[i], a[j]);
            }
        }
    }
}

void bubbleSort(vector<float>& a) {
    bool sorted;
    for (uint i = 0; i < a.size() - 1; ++i) {
        sorted = true;
        for (uint j = 0; j < a.size() - i - 1; ++j) {
            if (a[j] > a[j+1]) {
                sorted = false;
                swap(a[j], a[j+1]);
            }
        }
        if (sorted) break;
    }
}

void selectionSort(vector<float>& a) {
    uint m;
    for (uint i = 0; i < a.size() - 1; ++i) {
        m = i;
        for (uint j = i + 1; j < a.size(); ++j) {
            if (a[j] < a[m]) {
                m = j;
            }
        }
        if (m != i) {
            swap(a[i], a[m]);
        }
    }
}

void insertionSort(vector<float>& a) {
  for (uint i = 1; i < a.size(); ++i) {
    float e = a[i];
    uint j = i;
    for (; j >= 1 && a[j - 1] > e; --j) {
      a[j] = a[j - 1];
    }
    if (j != i) a[j] = e;
  }
}

void shellSort(vector<float>& a) {
    for (uint gap = a.size() / 2; gap > 0; gap /= 2) {
        for (uint i = gap; i < a.size(); ++i) {
            float e = a[i];
            uint j = i;
            for (; j >= gap && a[j - gap] > e; j -= gap) {
                a[j] = a[j - 1];
            }
            if (j != i) a[j] = e;
        }
    }
}

void siftDown(vector<float>& a, int i, int end) {
    while (i*2 + 1 <= end) {
        auto child = i*2 + 1;
        auto rep = i;
        if (a[rep] < a[child])                      rep = child;
        if (child+1 <= end && a[rep] < a[child+1])  rep = child+1;
        if (rep == i) return;
        swap(a[i], a[rep]);
        i = rep;
    }
}

void heapify(vector<float>& a) {
    for (int i = (a.size()-1 - 1) / 2; i >= 0; i--) {
        siftDown(a, i, a.size()-1);
    }
}

void heapSort(vector<float>& a) {
    heapify(a);
    for (int end = a.size() - 1; end > 0; --end) {
        swap(a[0], a[end]);
        siftDown(a, 0, end-1);
    }
}

void merge(vector<float>& a, uint lo, uint hi) {
    auto mid = (lo + hi) / 2;
    auto i = lo;
    auto j = mid + 1;
    vector<float> temp(hi - lo + 1);
    for (uint k = 0; k < temp.size(); ++k) {
        temp[k] = (i > mid || (j <= hi && a[j] < a[i]))? a[j++] : a[i++];
    }
    for (uint k = 0; k < temp.size(); ++k) {
        a[lo + k] = temp[k];
    }
}

void msort(vector<float>& a, uint lo, uint hi) {
    if (lo < hi) {
        auto mid = (lo + hi) / 2;
        msort(a, lo, mid);
        msort(a, mid+1, hi);
        merge(a, lo, hi);
    }
}

void mergeSort(vector<float>& a) {
    msort(a, 0, a.size() - 1);
}

int pivot(vector<float>& a, int lo, int hi) {
    float pivot = a[lo];
    int i = lo;
    int j = hi;
    for (;;) {
        while (i <= j && a[i] <= pivot) ++i;
        while (i <= j && a[j] >= pivot) --j;
        if (j < i) {
            swap(a[lo], a[j]);
            return j;
        }
        swap(a[i++], a[j--]);
    }
}

void quick(vector<float>& a, int lo, int hi) {
    if (lo < hi) {
        auto p = pivot(a, lo, hi);
        quick(a, lo, p-1);
        quick(a, p+1, hi);
    }
}

void quickSort(vector<float>& a) {
    quick(a, 0, a.size() - 1);
}

void testSort(void (*sort)(vector<float>&), void (*make)(vector<float>&),
              int n, string name) {
    vector<float> a(n);
    make(a);
    steady_clock::time_point begin = steady_clock::now();
    sort(a);
    steady_clock::time_point end = steady_clock::now();

    cout << (isSorted(a)? "sorted" : "not sorted");
    cout << setw(16) << name << " = "
         << duration_cast<nanoseconds>(end - begin).count() / 1e9
         << " seconds" << endl;
}

int main() {
    int n = 50000;

    cout << "Normal random array\n";
    testSort(exchangeSort,  normalRandom,   n, "Exchange sort");
    testSort(bubbleSort,    normalRandom,   n, "Bubble sort");
    testSort(selectionSort, normalRandom,   n, "Selection sort");
    testSort(insertionSort, normalRandom,   n, "Insertion sort");
    testSort(shellSort,     normalRandom,   n, "Shell sort");
    testSort(heapSort,      normalRandom,   n, "Heap sort");
    testSort(mergeSort,     normalRandom,   n, "Merge sort");
    testSort(quickSort,     normalRandom,   n, "Quick sort");

    cout << "Reversed arrays\n";
    testSort(exchangeSort,  reverse,   n, "Exchange sort");
    testSort(bubbleSort,    reverse,   n, "Bubble sort");
    testSort(selectionSort, reverse,   n, "Selection sort");
    testSort(insertionSort, reverse,   n, "Insertion sort");
    testSort(shellSort,     reverse,   n, "Shell sort");
    testSort(heapSort,      reverse,   n, "Heap sort");
    testSort(mergeSort,     reverse,   n, "Merge sort");
    testSort(quickSort,     reverse,   n, "Quick sort");

    cout << "Few Unique\n";
    testSort(exchangeSort,  fewUnique,   n, "Exchange sort");
    testSort(bubbleSort,    fewUnique,   n, "Bubble sort");
    testSort(selectionSort, fewUnique,   n, "Selection sort");
    testSort(insertionSort, fewUnique,   n, "Insertion sort");
    testSort(shellSort,     fewUnique,   n, "Shell sort");
    testSort(heapSort,      fewUnique,   n, "Heap sort");
    testSort(mergeSort,     fewUnique,   n, "Merge sort");
    testSort(quickSort,     fewUnique,   n, "Quick sort");

    cout << "Nearly sorted\n";
    testSort(exchangeSort,  almostSorted,   n, "Exchange sort");
    testSort(bubbleSort,    almostSorted,   n, "Bubble sort");
    testSort(selectionSort, almostSorted,   n, "Selection sort");
    testSort(insertionSort, almostSorted,   n, "Insertion sort");
    testSort(shellSort,     almostSorted,   n, "Shell sort");
    testSort(heapSort,      almostSorted,   n, "Heap sort");
    testSort(mergeSort,     almostSorted,   n, "Merge sort");
    testSort(quickSort,     almostSorted,   n, "Quick sort");

    return 0;
}

Overwriting 1.cpp


In [48]:
!g++ -std=c++17 -Wall 1.cpp && ./a.out

Normal random array
sorted   Exchange sort = 23.118 seconds
sorted     Bubble sort = 23.5983 seconds
sorted  Selection sort = 12.2095 seconds
sorted  Insertion sort = 5.00858 seconds
sorted      Shell sort = 0.0119634 seconds
sorted       Heap sort = 0.0268621 seconds
sorted      Merge sort = 0.0314463 seconds
sorted      Quick sort = 0.0114972 seconds
Reversed arrays
sorted   Exchange sort = 25.6854 seconds
sorted     Bubble sort = 26.2899 seconds
sorted  Selection sort = 9.45148 seconds
sorted  Insertion sort = 10.0072 seconds
sorted      Shell sort = 0.00977408 seconds
sorted       Heap sort = 0.023092 seconds
sorted      Merge sort = 0.0229287 seconds
sorted      Quick sort = 3.89576 seconds
Few Unique
sorted   Exchange sort = 9.15685 seconds
sorted     Bubble sort = 23.1108 seconds
sorted  Selection sort = 9.17317 seconds
sorted  Insertion sort = 4.74098 seconds
sorted      Shell sort = 0.0101011 seconds
sorted       Heap sort = 0.023186 seconds
sorted      Merge sort = 0.0268555 